<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with a custom metrics provider

This notebook demonstrates how to configure the custom monitor and custom metrics deployment by using IBM Watson OpenScale. It requires service credentials for the following services:
  * Watson OpenScale
  * Watson Machine Learning

  
## Contents

This notebook contains the following parts:

  1. [Set up your environment.](#setup)
  1. [Create the custom metrics provider - python function.](#provider)
  1. [Register the custom metrics provider and create a deployment.](#deployment)
  1. [Configure Watson OpenScale](#config)
  1. [Create the integrated system for the custom metrics provider.](#custom)
  1. [Set up the custom monitor definition and instance.](#instance)
  1. [Run the custom monitor](#run)


## 1. Set up your environment. <a name="setup"></a>

Before you use the sample code in this notebook, you must perform the following setup tasks:

### Install the  `ibm-watson-machine-learning` and `ibm-watson-openscale` packages.

In [ ]:
!pip install --upgrade ibm-watson-machine-learning | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

### Action: restart the kernel!

### Credentials for IBM Cloud Pak for Data
To authenticate, in the following code boxes, replace the sample data with your own credentials. Get the information from your system administrator or through the Cloud Pak for Data dashboard.


### Obtaining your Watson OpenScale credentials

You can retrieve the URL by running the following command: `oc get route -n namespace1 --no-headers | awk '{print $2}'` Replace the `namespace1` variable with your namespace.

You should have been assigned a username and password when you were added to the Cloud Pak for Data system. You might need to ask either your database administrator or your system administrator for some of the information.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
############################################################################################
# Paste your Watson OpenScale credentials into the following section and then run this cell.
############################################################################################
WOS_CREDENTIALS = {
    "url": "https://cpd-cpd.apps.ocp-675000m8qo-e56y.cloud.techzone.ibm.com",
    "username": "admin",
    "apikey": "X6JqihJv2VobrerV5G7Phvm322evuy7QNTwBVlLg"
}


### Enter your Watson OpenScale GUID.

For most systems, the default GUID is already entered for you. You would only need to update this particular entry if the GUID was changed from the default.


In [ ]:
from ibm_watson_openscale.utils.client_utils import get_my_instance_ids

my_instances = get_my_instance_ids(WOS_CREDENTIALS['url'], WOS_CREDENTIALS['username'], None, WOS_CREDENTIALS['apikey'], None)
my_instances

In [ ]:
#Update your Watson OpenScale datamart id.
WOS_GUID = "00000000-0000-0000-0000-000000000000"
data_mart_id = WOS_GUID

In [ ]:
WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS['instance_id'] = 'openshift'
WML_CREDENTIALS['version'] = '4.0'
WML_CREDENTIALS

### Define the Watson OpenScale subscription to which the custom metrics have to be sent.

Create a subscription from the Watson Openscale UI or SDK to configure custom metrics. You can configure custom metrics for the subscriptions that have predefined monitors, such as fairness, quality, or drift or without predefined monitors.

In [ ]:
#Configure Open Scale
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import APIClient
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

In [ ]:
authenticator = CloudPakForDataAuthenticator(
    url = WOS_CREDENTIALS['url'],
    username = WOS_CREDENTIALS['username'],
    apikey = WOS_CREDENTIALS['apikey'],
    disable_ssl_verification = True
)

wos_client = APIClient(service_url = WOS_CREDENTIALS['url'], authenticator = authenticator, service_instance_id = WOS_GUID)
wos_client.version

In [ ]:
wos_client.subscriptions.show()

In [ ]:
####################################################################
# Paste your Subscription in the following field and then run this cell.
####################################################################

subscription_id = '855685d5-b9f2-40b7-8dd0-ac492124fdcf'

In [ ]:
wos_client.monitor_definitions.show()

In [ ]:
#wml_python_function
parms = {
    "url": WOS_CREDENTIALS["url"],
    "username": WOS_CREDENTIALS["username"],
    "apikey": WOS_CREDENTIALS["apikey"]
}

parms

### Python function details

In [ ]:
PYTHON_FUNCTION_NAME = 'Veritas Toolkit Metrics Provider PreProd'
DEPLOYMENT_NAME = 'Veritas Toolkit Metrics Provider Deployment PreProd'

### OpenScale Custom Metrics Provider name

In [ ]:
CUSTOM_METRICS_PROVIDER_NAME = "Veritas Toolkit Metrics Provider PreProd"

### OpenSale Custom Monitor name

In [ ]:
###################################################################
# UPDATE your custom monitor name in the following field and then run this cell.
####################################################################
CUSTOM_MONITOR_NAME_LIST = [
    'Veritas_Toolkit_Metrics_Fairness_Sex',
    'Veritas_Toolkit_Metrics_Fairness_Marriage',
    'Veritas_Toolkit_Metrics_Performance'
]

## 2. Create the custom metrics provider - Python function. <a name="provider"></a>

The Python function receives the required variables, such as the `datamart_id`, `monitor_id`, `monitor_instance_id`, `monitor_run_id`, `monitor_instance_parameters` and `subscription_id` from the Watson OpenScale service when it is invoked by the custom monitor. 

In the Python function, add your own logic to compute the custom metrics in the `get_metrics` method, publish the metrics to the Watson Openscale service and update the status of the run to the `finished` state in the custom monitor instance run.

Update the `WOS_CREDENTIALS` in the Python function. 

In [ ]:
#wml_python_function
parms = {
    "url": WOS_CREDENTIALS["url"],
    "username": WOS_CREDENTIALS["username"],
    "apikey": WOS_CREDENTIALS["apikey"]
}

def custom_metrics_provider(parms = parms):
    from veritastool.model.model_container import ModelContainer
    from veritastool.usecases.credit_scoring import CreditScoring
    from io import BytesIO
    import pickle
    import numpy as np
    from sklearn.linear_model import LogisticRegression

    from ibm_watson_studio_lib import access_project_or_space
    import json
    import requests
    import base64
    from requests.auth import HTTPBasicAuth
    import time
    import uuid
    import datetime

    headers = {}
    headers["Content-Type"] = "application/json"
    headers["Accept"] = "application/json"

    #Get the access token
    def get_access_token():
        url = '{}/icp4d-api/v1/authorize'.format(parms['url'])
        payload = {
            'username': parms['username'],
            'api_key': parms['apikey']
        }
        response = requests.post(url, headers=headers, json=payload, verify=False)
        json_data = response.json()
        access_token = json_data['token']

        return access_token

    #Update the run status to Finished in the Monitor Run
    def update_monitor_run_status(base_url, access_token, custom_monitor_instance_id, run_id, status, error_msg = None):
        monitor_run_url = base_url + '/v2/monitor_instances/' + custom_monitor_instance_id + '/runs/'+run_id
        completed_timestamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        patch_payload  = []
        base_path = "/status"

        patch_payload.append(get_patch_request_field(base_path, "state", status))
        patch_payload.append(get_patch_request_field(base_path, "completed_at", completed_timestamp))

        if error_msg != None:
            error_json = get_error_json(error_msg)
            patch_payload.append(get_patch_request_field(base_path, "failure", error_json))

        headers["Authorization"] = "Bearer {}".format(access_token)
        response = requests.patch(monitor_run_url, headers=headers, json = patch_payload, verify=False)
        monitor_run_response = response.json()

        return response.status_code, monitor_run_response

    def get_error_json(error_message):
        trace = str(uuid.uuid4())
        error_json = {
            'trace': trace,
            'errors': [{
                'code': "custom_metrics_error_code",
                'message': str(error_message)
            }]
        }

        return error_json

    def get_patch_request_field(base_path, field_name, field_value, op_name="replace"):
        field_json = {
            "op": op_name,
            "path": "{0}/{1}".format(base_path, field_name),
            "value": field_value
        }

        return field_json

    #Add your code to compute the custom metrics
    def get_metrics(metric_type, protected_variable=None):
        #Get the model file
        wslib = access_project_or_space()
        raw_data_1 = wslib.load_data('credit_score_dict.pickle')

        #Load Credit Scoring Test Data
        readrawdata = raw_data_1.read()
        cs = pickle.load(BytesIO(readrawdata))

        #Model Container Parameters
        y_true = np.array(cs["y_test"])
        y_pred = np.array(cs["y_pred"])
        y_train = np.array(cs["y_train"])
        p_grp = {'SEX': [1], 'MARRIAGE':[1]}
        up_grp = {'SEX': [2], 'MARRIAGE':[2]}
        x_train = cs["X_train"]
        x_test = cs["X_test"]
        model_name = "credit_scoring"
        model_type = "classification"
        y_prob = cs["y_prob"]
        model_obj = LogisticRegression(C=0.1)
        model_obj.fit(x_train, y_train) #fit the model as required for transparency analysis

        
        #Create Model Container 
        container = ModelContainer(y_true, p_grp, model_type, model_name, y_pred, y_prob, y_train, x_train=x_train, \
                                   x_test=x_test, model_object=model_obj, up_grp=up_grp)
        
        #Create Use Case Object
        cre_sco_obj= CreditScoring(model_params = [container], fair_threshold = 80, fair_concern = "eligible", \
                                   fair_priority = "benefit", fair_impact = "normal", perf_metric_name="accuracy", \
                                   tran_row_num = [20,40], tran_max_sample = 1000, tran_pdp_feature = ['LIMIT_BAL'], tran_max_display = 10)

        cre_sco_obj.evaluate()

        dict_metric = {}

        if metric_type == 'Fairness':
            dict_metric = cre_sco_obj.get_fair_metrics_results()
        elif metric_type == 'Performance':
            dict_metric = cre_sco_obj.get_perf_metrics_results()

        if protected_variable:
            dict_metric = dict_metric[protected_variable]

        d = {}

        for k, v in dict_metric.items():
            d[k] = v[0]

        return d

    #Publish the Custom Metrics to OpenScale
    def publish_metrics(base_url, access_token, data_mart_id, subscription_id, custom_monitor_id, custom_monitor_instance_id, custom_monitoring_run_id, timestamp):
        #Generate an monitoring run id, where the publishing happens against this run id
        custom_metrics = {}

        if custom_monitor_id == 'Veritas_Toolkit_Metrics_Fairness_Sex'.lower():
            custom_metrics = get_metrics('Fairness', 'SEX')
        elif custom_monitor_id == 'Veritas_Toolkit_Metrics_Fairness_Marriage'.lower():
            custom_metrics = get_metrics('Fairness', 'MARRIAGE')
        elif custom_monitor_id == 'Veritas_Toolkit_Metrics_Performance'.lower():
            custom_metrics = get_metrics('Performance')

        measurements_payload = [
            {
                "timestamp": timestamp,
                "run_id": custom_monitoring_run_id,
                "metrics": [custom_metrics]
            }
        ]
        headers["Authorization"] = "Bearer {}".format(access_token)
        measurements_url = base_url + '/v2/monitor_instances/' + custom_monitor_instance_id + '/measurements'
        response = requests.post(measurements_url, headers=headers, json = measurements_payload, verify=False)
        published_measurement = response.json()

        return response.status_code, published_measurement

    def publish(input_data):
        timestamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ")

        payload = input_data.get("input_data")[0].get("values")
        data_mart_id = payload['data_mart_id']
        subscription_id = payload['subscription_id']
        custom_monitor_id = payload['custom_monitor_id']
        custom_monitor_instance_id = payload['custom_monitor_instance_id']
        custom_monitor_instance_params  = payload['custom_monitor_instance_params']
        custom_monitor_run_id = payload['custom_monitor_run_id']

        base_url = parms['url'] + '/openscale' + '/' + data_mart_id
        access_token = get_access_token()

        published_measurements = []
        error_msgs = []
        run_status = "finished"

        try:
            last_run_time = custom_monitor_instance_params.get("last_run_time")
            max_records = custom_monitor_instance_params.get("max_records")
            min_records = custom_monitor_instance_params.get("min_records")
            error_msg = None
            status_code, published_measurement = publish_metrics(base_url, access_token, data_mart_id, subscription_id, custom_monitor_id, custom_monitor_instance_id, custom_monitor_run_id, timestamp)

            if int(status_code) in [200, 201, 202]:
                published_measurements.append(published_measurement)
            else:
                run_status = "error"
                error_msg = published_measurement
                error_msgs.append(error_msg)

            # update the monitor run status to finished/error
            status_code, response = update_monitor_run_status(base_url, access_token, custom_monitor_instance_id, custom_monitor_run_id, run_status, error_msg)

            if not int(status_code) in [200, 201, 202]:
                error_msgs.append(response)

        except Exception as ex:
            error_msgs.append(str(ex))

        if len(error_msgs) == 0:
            response_payload = {
                "predictions" : [{
                    "values" : published_measurements
                }]
            }
        else:
            response_payload = {
                "predictions":[],
                "errors": error_msgs
            }

        return response_payload

    return publish

## 3. Register the custom metrics provider and create a deployment. <a name="deployment"></a>

In [ ]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

In [ ]:
wml_client.spaces.list(limit=10)

In [ ]:
space_id = "6af5797b-d44c-4c94-b4a2-0e1b57bc108a" #update your space id
wml_client.set.default_space(space_id)

### Remove existing function and deployment.

In [ ]:
deployments_list = wml_client.deployments.get_details()

In [ ]:
deployments_list

In [ ]:
deployments_list = wml_client.deployments.get_details()

for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    deployment_id = deployment["metadata"]["id"]
    
    if deployment["metadata"]["name"] == DEPLOYMENT_NAME:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)

wml_client.repository.list_functions()

### Create the function meta properties.


In [ ]:
software_spec_id =  wml_client.software_specifications.get_id_by_name('MAS_Veritas_Custom_Environment')
print(software_spec_id)
function_meta_props = {
    wml_client.repository.FunctionMetaNames.NAME: PYTHON_FUNCTION_NAME,
    wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: software_spec_id
}

### Store the Python function.

In [ ]:
function_artifact = wml_client.repository.store_function(meta_props=function_meta_props, function=custom_metrics_provider)
function_uid = wml_client.repository.get_function_id(function_artifact)
print("Function UID = " + function_uid)

In [ ]:
function_details = wml_client.repository.get_details(function_uid)
from pprint import pprint
pprint(function_details)

### Deploy the Python function.


In [ ]:
hardware_spec_id = wml_client.hardware_specifications.get_id_by_name('M')
hardware_spec_id

### Create deployment metadata for the Python function.

In [ ]:
deploy_meta = {
    wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {},
    wml_client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"id": hardware_spec_id}
}

### Create a deployment.

In [ ]:
deployment_details = wml_client.deployments.create(function_uid, meta_props=deploy_meta)

### Get the scoring URL.


In [ ]:
created_at = deployment_details['metadata']['created_at']
find_string_pos = created_at.find("T")

if find_string_pos != -1:
    current_date = created_at[0:find_string_pos]

    scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
scoring_url = scoring_url + "?version=" + current_date
print(scoring_url)

## 4. Configure OpenScale. <a name="config"></a>

Import the required libraries and set up the Watson OpenScale Python client.

In [ ]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMetricRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MetricThreshold
from ibm_watson_openscale.supporting_classes.enums import MetricThresholdTypes
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorTagRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Target
from ibm_watson_openscale.supporting_classes.enums import TargetTypes
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import IntegratedSystems
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorInstanceSchedule, ScheduleStartTime, MonitorRuntime

from datetime import datetime, timezone, timedelta
import uuid

In [ ]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = CloudPakForDataAuthenticator(
    url = WOS_CREDENTIALS['url'],
    username = WOS_CREDENTIALS['username'],
    apikey = WOS_CREDENTIALS['apikey'],
    disable_ssl_verification=True
)
wos_client = APIClient(service_url=WOS_CREDENTIALS['url'], authenticator=authenticator, service_instance_id=WOS_GUID)
wos_client.version

## 5. Create the integrated system for the custom metrics provider. <a name="custom"></a>


Update the custom metrics deployment URL, which is created during the Python function creation in the integrated system. Watson OpenScale invokes the deployment URL at runtime to compute the custom metrics. 

You must define the authentication type based on the communication with custom metrics deployment. Watson OpenScale supports 2 types of authentication: basic and bearer. If custom metrics deployment accepts the `basic` authentication type, then provide `auth_type=basic` otherwise use `auth_type=bearer`.

In [ ]:
CUSTOM_METRICS_PROVIDER_CREDENTIALS = {
    "auth_type":"bearer",
    "token_info": {
        "url": "{}/icp4d-api/v1/authorize".format(WOS_CREDENTIALS['url']),
        "headers": {
            "Content-Type": "application/json",
            "Accept": "application/json"
        },
        "payload": {
            "username": WOS_CREDENTIALS['username'],
            "api_key": WOS_CREDENTIALS['apikey'],
        },
        "method": "post"
    }
}

### Remove existing integrated system 

In [ ]:
# Delete existing custom metrics provider integrated systems if present
integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems

for system in integrated_systems:
    if system.entity.type == 'custom_metrics_provider' and system.entity.name == CUSTOM_METRICS_PROVIDER_NAME:
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

In [ ]:
custom_metrics_integrated_system = IntegratedSystems(wos_client).add(
    name=CUSTOM_METRICS_PROVIDER_NAME,
    description=CUSTOM_METRICS_PROVIDER_NAME,
    type="custom_metrics_provider",
    credentials=CUSTOM_METRICS_PROVIDER_CREDENTIALS,
    connection={
        "display_name": CUSTOM_METRICS_PROVIDER_NAME,
        "endpoint": scoring_url
    }
).result

integrated_system_id = custom_metrics_integrated_system.metadata.id
print(custom_metrics_integrated_system)

## 6. Set up the custom monitor definition and instance. <a name="instance"></a>


### Check for the existence of the custom monitor definition.


In [ ]:
def get_custom_monitor_definition(custom_monitor_name):
    monitor_definitions = wos_client.monitor_definitions.list().result.monitor_definitions
    for definition in monitor_definitions:
        if custom_monitor_name == definition.entity.name:
            return definition
    return None

### Create the  custom monitor definition.

Update the custom metric names, threshold types (`LOWER_LIMIT`, `UPPER_LIMIT`) and default values as required. You can define the threshold type as lower limit, upper limit, or both.

In [ ]:
###################################################################
# Update your custom monitor metrics names in the following field. Use the same metric names for creating the 
# monitor definition and publishing the metrics to Openscale in your python function
####################################################################
VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES = [
    'demographic_parity',
    'equal_opportunity',
    'fpr_parity',
    'tnr_parity',
    'fnr_parity',
    'ppv_parity',
    'npv_parity',
    'fdr_parity',
    'for_parity',
    'equal_odds',
    'neg_equal_odds',
    'calibration_by_group',
    'auc_parity',
    'log_loss_parity',
    'mi_independence',
    'mi_separation',
    'mi_sufficiency'
]

VERITAS_TOOLKIT_METRICS_PERFORMANCE_NAMES = [
    'selection_rate',
    'accuracy',
    'balanced_acc',
    'recall',
    'precision',
    'f1_score',
    'tnr',
    'fnr',
    'npv',
    'roc_auc',
    'log_loss'
]

#Update the tag values if you want to fetch the metrics by tags
TAGS = ['veritas']
TAG_DESCRIPTION = ['Veritas Toolkit Metrics']

### Create  schedule for custom monitor

Creates a schedule for custom monitor with a defined repeat type and repeat interval. Supported repeat types are "minute", "hour", "day", "week", "month", "year". Defaults to once every hour if `repeat_interval` and  `repeat_type` values are not specified


In [ ]:
#Enable the schedule for your custom monitor. Update the value to false if you want to disable the schedule
ENABLE_SCHEDULE = True

#Update the repeat interval and repeat type: Default is 1 hour
repeat_interval = 1
repeat_type = "hour"

#Update the delay unit and interval to trigger the first schedule run after the custom monitor instance is created. Default is 30 minutes
delay_unit = "minute"
delay_time= 30

start_time= ScheduleStartTime(type = "relative", delay_unit = delay_unit, delay = delay_time)
schedule = MonitorInstanceSchedule(repeat_interval=repeat_interval,repeat_unit=repeat_type, start_time=start_time)

In [ ]:
#Update the Threshold types and default values of the metrics
def veritas_toolkit_metrics_fairness_definitions():
    metrics = [
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[0], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[1], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[2], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[3], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[4], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[5], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[6], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[7], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[8], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[9], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[10], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[11], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[12], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[13], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[14], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[15], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_FAIRNESS_NAMES[16], thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-0.43), MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.43)])
    ]

    tags = [MonitorTagRequest(name=TAGS[0], description=TAG_DESCRIPTION[0])]
    
    return metrics, tags

def veritas_toolkit_metrics_performance_definitions():
    metrics = [
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_PERFORMANCE_NAMES[0]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_PERFORMANCE_NAMES[1]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_PERFORMANCE_NAMES[2]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_PERFORMANCE_NAMES[3]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_PERFORMANCE_NAMES[4]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_PERFORMANCE_NAMES[5]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_PERFORMANCE_NAMES[6]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_PERFORMANCE_NAMES[7]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_PERFORMANCE_NAMES[8]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_PERFORMANCE_NAMES[9]),
        MonitorMetricRequest(name=VERITAS_TOOLKIT_METRICS_PERFORMANCE_NAMES[10])
   ]

    tags = [MonitorTagRequest(name=TAGS[0], description=TAG_DESCRIPTION[0])]
    
    return metrics, tags

In [ ]:
def create_custom_monitor_definition(custom_monitor_name):
    #Check if the custom monitor definition already exists or not
    existing_definition = get_custom_monitor_definition(custom_monitor_name)

    #If it does not exists, then create a new one.
    if existing_definition is None:
        if 'Fairness' in custom_monitor_name:
            metrics, tags = veritas_toolkit_metrics_fairness_definitions()
        elif 'Performance' in custom_monitor_name:
            metrics, tags = veritas_toolkit_metrics_performance_definitions()
        
        if ENABLE_SCHEDULE:
            monitor_runtime = MonitorRuntime(type="custom_metrics_provider")
            custom_monitor_details = wos_client.monitor_definitions.add(name=custom_monitor_name, metrics=metrics, tags=tags, schedule = schedule, monitor_runtime = monitor_runtime, background_mode=False).result
        else:
            custom_monitor_details = wos_client.monitor_definitions.add(name=custom_monitor_name, metrics=metrics, tags=tags, background_mode=False).result
    else:
        #Otherwise, send the existing definition
        custom_monitor_details = existing_definition
    
    return custom_monitor_details

In [ ]:
CUSTOM_MONITOR_ID_LIST = []

for custom_monitor_name in CUSTOM_MONITOR_NAME_LIST:
    custom_monitor_details = create_custom_monitor_definition(custom_monitor_name)
    print(custom_monitor_details)
    custom_monitor_id = custom_monitor_details.metadata.id
    CUSTOM_MONITOR_ID_LIST.append(custom_monitor_id)

CUSTOM_MONITOR_ID_LIST

### Check the existence of custom monitor instance.


In [ ]:
def get_custom_monitor_instance(custom_monitor_id):
    monitor_instances = wos_client.monitor_instances.list(data_mart_id = WOS_GUID, monitor_definition_id = custom_monitor_id, target_target_id = subscription_id).result.monitor_instances
    if len(monitor_instances) == 1:
        return monitor_instances[0]
    return None

In [ ]:
# Openscale MRM service invokes custom metrics deployment url during runtime and wait for the default time of 60 second's to 
# to check the run status ie finished/Failed and fetch the latest measurement. Increase the wait time, if the runtime deployment 
# takes more than 60 seconds to compute and publish the custom metrics 

#Update the wait time here.
custom_metrics_wait_time = 180 #time in seconds <update the time here>

#Maximum number of records to consider during custom monitor run evaluation. Update max_records value as per the requirement
max_records = None
#Minimum number of records to consider during custom monitor run evaluation. Update min_records value as per the requirement
min_records = None

### Update the custom monitor instance.

In [ ]:
def update_custom_monitor_instance(custom_monitor_instance_id):
    payload = [{
        "op": "replace",
        "path": "/parameters",
        "value": {
            "custom_metrics_provider_id": integrated_system_id,
            "custom_metrics_wait_time": custom_metrics_wait_time,
            "enable_custom_metric_runs": True
        }
    }]
    
    if max_records is not None:
        payload[0]["value"]["max_records"] = max_records    
    if min_records is not None:
        payload[0]["value"]["min_records"] = min_records 
        
    response = wos_client.monitor_instances.update(custom_monitor_instance_id, payload, update_metadata_only = True)
    result = response.result
    
    return result

### For the custom monitor definition, create a custom monitor instance.


In [ ]:
def create_custom_monitor_instance(custom_monitor_id):
    #Check if an custom monitor instance already exists
    existing_monitor_instance = get_custom_monitor_instance(custom_monitor_id)

    #If it does not exist, then create one
    if existing_monitor_instance is None:
        target = Target(
            target_type=TargetTypes.SUBSCRIPTION,
            target_id=subscription_id
        )
        parameters = {
            "custom_metrics_provider_id": integrated_system_id,
            "custom_metrics_wait_time": custom_metrics_wait_time,
            "enable_custom_metric_runs": True
        }
        
        if max_records is not None:
            parameters["max_records"] = max_records
        if min_records  is not None:
            parameters["min_records"] = min_records 
        
        #Create the custom monitor instance id here.
        custom_monitor_instance_details = wos_client.monitor_instances.create(
            data_mart_id=WOS_GUID,
            background_mode=False,
            monitor_definition_id=custom_monitor_id,
            target=target,
            parameters=parameters
        ).result
    else:
        #Otherwise, update the existing one with latest integrated system details.
        instance_id = existing_monitor_instance.metadata.id
        custom_monitor_instance_details = update_custom_monitor_instance(instance_id)
    
    return custom_monitor_instance_details

In [ ]:
CUSTOM_MONITOR_INSTANCE_ID_LIST = []

for custom_monitor_id in CUSTOM_MONITOR_ID_LIST:
    monitor_instance_details = create_custom_monitor_instance(custom_monitor_id)
    custom_monitor_instance_id = monitor_instance_details.metadata.id
    print(monitor_instance_details)
    CUSTOM_MONITOR_INSTANCE_ID_LIST.append(custom_monitor_instance_id)

CUSTOM_MONITOR_INSTANCE_ID_LIST

In [ ]:
wos_client.monitor_instances.show(data_mart_id = WOS_GUID, target_target_id = subscription_id)

### Run the custom monitor <a name="run"></a>


In [ ]:
#Execute the custom metrics provider deployment
monitor_instance_run_info = wos_client.monitor_instances.run(
    background_mode = False,
    monitor_instance_id = 'ffabd068-5d67-494e-96f3-f420df478627'
).result

monitor_instance_run_info
custom_monitor_run_id = monitor_instance_run_info.metadata.id

In [ ]:
#Execute the custom metrics provider deployment
monitor_instance_run_info = wos_client.monitor_instances.run(
    background_mode = False,
    monitor_instance_id = 'cdcfcd77-2e13-43d0-b911-60bcb363467d'
).result

monitor_instance_run_info
custom_monitor_run_id = monitor_instance_run_info.metadata.id

In [ ]:
#Execute the custom metrics provider deployment
monitor_instance_run_info = wos_client.monitor_instances.run(
    background_mode = False,
    monitor_instance_id = '3cff16ba-9d79-4e6d-9d00-a083d5521853'
).result

monitor_instance_run_info
custom_monitor_run_id = monitor_instance_run_info.metadata.id

### Show Custom Metrics

In [ ]:
wos_client.monitor_instances.show_metrics(monitor_instance_id='ffabd068-5d67-494e-96f3-f420df478627')

In [ ]:
wos_client.monitor_instances.show_metrics(monitor_instance_id='cdcfcd77-2e13-43d0-b911-60bcb363467d')

In [ ]:
wos_client.monitor_instances.show_metrics(monitor_instance_id='3cff16ba-9d79-4e6d-9d00-a083d5521853')

In [ ]:
#Cleanup
wos_client.monitor_instances.delete(
    background_mode=False,
    monitor_instance_id='ceec3577-8499-4234-9c8f-1a297957a6d8'
)

wos_client.monitor_definitions.delete(monitor_definition_id='veritas_toolkit_fairness_metrics')

## Recap of the steps performed in this notebook

- Create a python function
- Deploy the python function to WML
- Create an OpenScale Integrated System pointing to the python function
- Create a Custom Monitor Definition mentioning various custom metrics
- Create a Custom Monitor Instance and specify the Integrated System ID in the monitor instance configuration.
- Run the Custom monitor

# [OPTIONAL STEP] Invoke the custom metrics deployment python function as part of this notebook.

Validate the custom metrics provider deployment by providing the correct set of paramaters to generate the custom metrics.


In [ ]:
import uuid

parameters = {
    "custom_metrics_provider_id": integrated_system_id,
    "custom_metrics_wait_time": custom_metrics_wait_time
}

if max_records is not None:
    parameters["max_records"] = max_records
if min_records is not None:
    parameters["min_records"] = min_records 

payload = {
    "data_mart_id" : WOS_GUID,
    "subscription_id" : subscription_id,
    "custom_monitor_id" : custom_monitor_id,
    "custom_monitor_instance_id" : custom_monitor_instance_id,
    "custom_monitor_run_id":custom_monitor_run_id,
    "custom_monitor_instance_params": parameters  
}

input_data = {"input_data": [{"values": payload}]}
input_data

In [ ]:
func_result = custom_metrics_provider()(input_data)
print(func_result)

In [ ]:
deployment_uid=wml_client.deployments.get_uid(deployment_details)
job_details = wml_client.deployments.score(deployment_uid, input_data)
pprint(job_details)

## Congratulations

You have finished configuring Custom Monitor Definition and Monitor instance and executing Custom Monitor Run for your Subscription. You can also run the custom monitor from `Watson OpenScale Dashboard`(https://url-to-your-cp4d-cluster/aiopenscale). Click the tile of your model and select `Evaluate Now` option from `Actions` drop down menu to run the custom monitor.